<a href="https://colab.research.google.com/github/Merna177/FashionMnist/blob/master/lesson2_fashionMinST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

0it [00:00, ?it/s]

26427392it [00:02, 10542480.59it/s]                             


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 71863.39it/s]                            
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:01, 3032987.52it/s]                            
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 24605.43it/s]            

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
from torch import nn, optim
import torch.nn.functional as F


In [0]:
class Network(nn.Module):
  def __init__(self):
        super().__init__()
        self.fc1= nn.Linear(784,256)
        self.fc2= nn.Linear(256,128)
        self.fc3=nn.Linear(128,64)
        self.fc4=nn.Linear(64,10)
  def forward(self,x):
     # make sure input tensor is flattened as x.shape[0] is batch size , -1 to flatten the remaining dimensions
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x
          

In [0]:
model = Network()
#i used NLL because i used in last layer log softmax activation function
criterion = nn.NLLLoss()
#adam is a modified version of gredient descent but has momentum to speed up the process
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [6]:
epochs = 30
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))


Epoch: 1/30..  Training Loss: 0.258..  Test Loss: 0.382..  Test Accuracy: 0.874
Epoch: 2/30..  Training Loss: 0.255..  Test Loss: 0.359..  Test Accuracy: 0.876
Epoch: 3/30..  Training Loss: 0.250..  Test Loss: 0.371..  Test Accuracy: 0.872
Epoch: 4/30..  Training Loss: 0.242..  Test Loss: 0.361..  Test Accuracy: 0.879
Epoch: 5/30..  Training Loss: 0.236..  Test Loss: 0.380..  Test Accuracy: 0.874
Epoch: 6/30..  Training Loss: 0.233..  Test Loss: 0.362..  Test Accuracy: 0.883
Epoch: 7/30..  Training Loss: 0.234..  Test Loss: 0.360..  Test Accuracy: 0.880
Epoch: 8/30..  Training Loss: 0.219..  Test Loss: 0.377..  Test Accuracy: 0.882
Epoch: 9/30..  Training Loss: 0.223..  Test Loss: 0.375..  Test Accuracy: 0.874
Epoch: 10/30..  Training Loss: 0.211..  Test Loss: 0.385..  Test Accuracy: 0.881
Epoch: 11/30..  Training Loss: 0.212..  Test Loss: 0.393..  Test Accuracy: 0.879
Epoch: 12/30..  Training Loss: 0.206..  Test Loss: 0.389..  Test Accuracy: 0.883
Epoch: 13/30..  Training Loss: 0.200.

In [0]:
torch.save(model.state_dict(), 'checkpoint.pth')